In [1]:
from utils import *

2022-10-21 01:23:18.802667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-21 01:23:18.921709: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-21 01:23:19.405169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moises/workspace/devel/lib
2022-10-21 01:23:19.405272: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.

In [2]:
tickers_path = 'data/tickers/sp500.json'
dst_path = 'data/stocks/'

In [3]:
success, fail = download_data(tickers_path, dst_path)
print(fail)

[]


In [32]:
src_path = 'data/stocks/'
dst_path = 'data/'

In [ ]:
merge_csv(src_path, dst_path)

In [2]:
df = pd.read_csv('data/dataset.csv')
df = df[-6000:-80]
df = df.dropna(axis=1)
df = df.set_index('Date')

In [3]:
dataset_path = 'data/dataset.csv'
data_size = 5000
input_tickers = df.columns
output_tickers = df.columns

In [4]:
scaler, x, y, dic = dataa(dataset_path, data_size, input_tickers, output_tickers, step_size=1, input_size=60, output_size=20, feature_range=(0,1))

In [5]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

In [6]:
x.shape, y.shape

((4920, 60, 356), (4920, 20, 356))

In [7]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info = [(50, 0.2, True), (70, 0.15, True), (70, 0.15, True), (60, 0.2, True), (40, 0.15, False)]

input_shape, output_shape

((60, 356), (20, 356))

In [9]:
model = create_model(input_shape, output_shape, layers_info)

In [10]:
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val, y_val)
)

2022-10-21 01:25:03.961328: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 294169920 exceeds 10% of free system memory.
2022-10-21 01:25:04.112804: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98056640 exceeds 10% of free system memory.


Epoch 1/100
108/108 [==============================] - 16s 107ms/step - loss: 0.0665 - accuracy: 0.0071 - val_loss: 0.0432 - val_accuracy: 0.0020
Epoch 2/100
108/108 [==============================] - 9s 88ms/step - loss: 0.0205 - accuracy: 0.0392 - val_loss: 0.0107 - val_accuracy: 0.0506
Epoch 3/100
108/108 [==============================] - 9s 83ms/step - loss: 0.0106 - accuracy: 0.0746 - val_loss: 0.0066 - val_accuracy: 0.1183
Epoch 4/100
108/108 [==============================] - 9s 85ms/step - loss: 0.0078 - accuracy: 0.1536 - val_loss: 0.0052 - val_accuracy: 0.1663
Epoch 5/100
108/108 [==============================] - 9s 87ms/step - loss: 0.0068 - accuracy: 0.1805 - val_loss: 0.0042 - val_accuracy: 0.1876
Epoch 6/100
108/108 [==============================] - 10s 94ms/step - loss: 0.0062 - accuracy: 0.2016 - val_loss: 0.0038 - val_accuracy: 0.2258
Epoch 7/100
108/108 [==============================] - 10s 92ms/step - loss: 0.0058 - accuracy: 0.2071 - val_loss: 0.0038 - val_accur

In [18]:
model_name = 'LSTM_test'
model.save('models/' + model_name + '/model')

INFO:tensorflow:Assets written to: models/LSTM_test/model/assets


INFO:tensorflow:Assets written to: models/LSTM_test/model/assets
